https://www.nltk.org/howto/lm.html

Essa é a versão final oficial

# Código Mestrado

https://stackabuse.com/python-for-nlp-working-with-facebook-fasttext-library/

https://github.com/PacktPublishing/fastText-Quick-Start-Guide/blob/master/chapter5/python%20fastText%20unsupervised%20learning.ipynb

## Bibliotecas

In [293]:
import pandas as pd
import re
from nltk.lm import Laplace
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter
import fasttext
import numpy as np
import itertools
import string
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import ngrams,bigrams, trigrams
from collections import Counter, defaultdict
import math
from fasttext import load_model
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import pad_both_ends

## Leitura de Base

### DataBase

In [294]:
papers = joblib.load('papers')

In [295]:
len(papers)

707

papers_treino = dict(itertools.islice(papers.items(),495))

In [296]:
papers_treino = dict(list(papers.items())[0:495])
papers_teste = dict(list(papers.items())[495:708])

### Padrão Ouro

In [297]:
padrao_ouro = joblib.load('selecao_padrao_ouro')

In [298]:
padrao_ouro_treino = dict(list(padrao_ouro.items())[0:495])
padrao_ouro_teste  = dict(list(padrao_ouro.items())[495:708])

In [299]:
len(padrao_ouro_treino)

495

pd.DataFrame.from_dict(padrao_ouro_treino.items()).to_excel('ouro_treino.xlsx', index=False)

# Pré-Processamento

In [300]:
def preprocessamento(corpus: str):
    """ Pré-processamento inicial para remover pontuações, padronizar a caixa e pegar o conteúdo de cada paper"""
    regIter = re.finditer('(<PAPER>(.*)<\/PAPER>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    #lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        #txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\\t","",txt)
        #txt = re.sub('\/','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        #txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;\?@\[\]^_`{|}\~\%\.]', '', txt)
        txt = re.sub('=', ' ', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        #txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        txt = re.sub('i\.e.', '', txt)
        txt = re.sub('acc\.', '', txt)
        txt = re.sub('e\.g\.', '', txt)
    return txt

In [301]:
def pre_processamento_ouro(txt):
    #txt = re.sub('\\n',' ', txt)
    txt = txt.lower()
    #txt = re.sub(r'<.*?>','', txt)
    txt = re.sub("\\t","",txt)
    #txt = re.sub('\/','', txt)
    txt = re.sub("\-","",txt)
    txt = txt.strip()
    txt = re.sub(' +', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub('&amp;quot;', '', txt)
    txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
    txt = re.sub('\.+', '', txt)
    txt = re.sub('[!"\#\$\%\&\'()\*\+,:;=\?@\[\]^_`{|}\~\%\.]', '', txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    #txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub('i\.e.', '', txt)
    txt = re.sub('acc\.', '', txt)
    txt = re.sub('e\.g\.', '', txt)
    return txt

In [302]:
papers_treino2 = {k: preprocessamento(v) for k, v in papers_treino.items()}
padrao_ouro_treino2 = {k: pre_processamento_ouro(v) for k, v in padrao_ouro_treino.items()}

In [303]:
papers_teste2 = {k: preprocessamento(v) for k, v in papers_teste.items()}
padrao_ouro_teste2 = {k: pre_processamento_ouro(v) for k, v in padrao_ouro_teste.items()}

In [304]:
papers_treino2[13]

'<s sid >a practical partofspeech tagger</s> <abstract> <s sid  ssid >we present an implementation of a partofspeech tagger based on a hidden markov model</s> <s sid  ssid >the methodology enables robust and accurate tagging with few resource requirements</s> <s sid  ssid >only a lexicon and some unlabeled training text are required</s> <s sid  ssid >accuracy exceeds </s> <s sid  ssid >we describe implementation strategies and optimizations which result in highspeed operation</s> <s sid  ssid >three applications for tagging are described phrase recognition word sense disambiguation and grammatical function assignment</s> <s sid  ssid > desiderata many words are ambiguous in their part of speech</s> <s sid  ssid >for example tag can be a noun or a verb</s> <s sid  ssid >however when a word appears in the context of other words the ambiguity is often reduced in a tag is a partofspeech label the tag can only be a noun</s> <s sid  ssid >a tagger is a system that uses context to assign part

# Word Embedding

## Pré-Processamento para Embedding

In [305]:
def separa_frases(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('</s>')
    return txt

In [306]:
def separa_frases_ouro(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('\n')
    return txt

In [307]:
def preprocessamento_secundario(txt: str):
    """Remoção das demais tags após quebrar no </s>"""
    txt = re.sub(r'<.*?>','', txt)
    txt = re.sub('\/','', txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

In [308]:
def preprocessamento_secundario_ouro(txt: str):
    """Remoção das demais tags após quebrar no \n"""
    txt = re.sub(r'<.*?>','', txt)
    txt = re.sub('\\n',' ', txt)
    txt = re.sub('\/','', txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

In [309]:
def segundo_preprocessamento(dicionario):
    """input: dicionário
        output: frases pré-processadas"""
    frases_papers_treino={}
    for i,v in dicionario.items():
        x=[]
        for j in v:
            j=preprocessamento_secundario(j)
            if j!='': #remove itens nulos da lista
                x.append(j)
        frases_papers_treino[i]= x
    return frases_papers_treino

frases_papers_treino={}
for i,v in papers_treino3.items():
    x=[]
    for j in v:
        j=preprocessamento_secundario(j)
        if j!='': #remove itens nulos da lista
            x.append(j)
    frases_papers_treino[i]= x

In [310]:
papers_treino3 = {k: separa_frases(v) for k, v in papers_treino2.items()}
frases_papers_treino = segundo_preprocessamento(papers_treino3)

In [311]:
papers_teste3 = {k: separa_frases(v) for k, v in papers_teste2.items()}
frases_papers_teste = segundo_preprocessamento(papers_teste3)

In [312]:
len(papers_teste3)

212

### Input Word2Vec

In [313]:
lista_tokens=[]
for i,v in frases_papers_treino.items():
    for j in v:
        lista_tokens.append(j.split(' '))

### Input FastText

In [314]:
with open('preprocessado_treino_fasttext.txt', 'w', encoding='utf-8') as f:
    for lista in frases_papers_treino.values():
        for line in lista:
            f.write(line)
            f.write('\n')

## Word2Vec

### CBOW

model_w2v_cbow = Word2Vec(lista_tokens, min_count=1, epochs=100, sg=0)

In [315]:
if os.path.exists('model_w2v_cbow'):
    model_w2v_cbow = joblib.load('model_w2v_cbow')
else:
    model_w2v_cbow = Word2Vec(lista_tokens, min_count=1, epochs=1000, sg=0)

In [316]:
len(model_w2v_cbow.wv.key_to_index)

46323

### SKIP-GRAM

model_w2v_sg = Word2Vec(lista_tokens, min_count=1, epochs=100, sg=1)

In [317]:
if os.path.exists('model_w2v_sg'):
    model_w2v_sg = joblib.load('model_w2v_sg')
else:
    model_w2v_sg = Word2Vec(lista_tokens, min_count=1, epochs=1000, sg=1)

In [318]:
len(model_w2v_sg.wv.key_to_index)

46323

## Fast Text

### CBOW

model_ft_cbow = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='cbow', minCount=1, epoch=1000)
model_ft_cbow.save_model('model_ft_cbow.bin')

In [319]:
if os.path.exists('model_ft_cbow.bin'):
    model_ft_cbow = load_model('model_ft_cbow.bin')
else:
    model_ft_cbow = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='cbow', minCount=1, epoch=1000)
    #model_ft_cbow.save_model('model_ft_cbow.bin')

In [320]:
len(model_ft_cbow.get_words( on_unicode_error='ignore'))

46324

if os.path.exists('model_ft_cbow.bin'):
    model_ft_cbow = load_model('model_ft_cbow.bin')
else:
    model_ft_cbow = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='cbow', minCount=1, epoch=100)
    model_ft_cbow.save_model('model_ft_cbow.bin')

### SKIP-GRAM

model_ft_sg = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='skipgram', minCount=1, epoch=1000)
model_ft_sg.save_model('model_ft_sg.bin')

In [321]:
if os.path.exists('model_ft_sg.bin'):
    model_ft_sg = load_model('model_ft_sg.bin')
else:
    model_ft_sg = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='skipgram', minCount=1, epoch=1000)
    #model_ft_sg.save_model('model_ft_sg.bin')

In [322]:
len(model_ft_sg.words)

46324

len(model_ft_sg.get_words( on_unicode_error='ignore'))

print(model_ft_sg.get_subwords('tnt'))

In [323]:
#comparando os tokens dos 2 embeddings
lista_w2v=[]
for i in model_w2v_sg.wv.key_to_index:
    lista_w2v.append(i)
    
lista_ft=[]
for i in model_ft_sg.get_words( on_unicode_error='ignore'):
    lista_ft.append(i)
    
list(set(lista_ft)-set(lista_w2v))

['</s>']

# Medida de Similaridade

## Detecção das Seções 
Abstract, Introdução e Conclusão

In [324]:
def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [325]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<section.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [326]:
def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<section.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<section.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<section.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    return textos

In [327]:
abstract        = {k: preprocessa_abstract(v) for k, v in papers_treino2.items()}
abstract_frases = {k: separa_frases(j) for k, v in abstract.items() for j in v}
abstract_frases = segundo_preprocessamento(abstract_frases)
abstract_palavras = {k:[word_tokenize(f) for f in v] for k,v in abstract_frases.items()}

In [328]:
introducao = {k: preprocessa_intro(v) for k, v in papers_treino2.items()}
introducao_frases = {k: separa_frases(j) for k, v in introducao.items() for j in v}
introducao_frases = segundo_preprocessamento(introducao_frases)
introducao_palavras = {k:[word_tokenize(f) for f in v] for k,v in introducao_frases.items()}

In [329]:
conclusao = {k: preprocessa_conclusion(v) for k, v in papers_treino2.items()}
conclusao_frases = {k: separa_frases(j) for k, v in conclusao.items() for j in v}
conclusao_frases = segundo_preprocessamento(conclusao_frases)
conclusao_palavras = {k:[word_tokenize(f) for f in v] for k,v in conclusao_frases.items()}

Validei que tudo mede 495

## Aplicação do Word Embedding

`model_w2v_cbow`

`model_w2v_sg`

`model_ft_cbow`

`model_ft_sg`

model_ft_sg.get_nearest_neighbors('method', k=5)

model_ft_sg.get_word_vector('method')

In [330]:
dicio={}
for k,v in abstract.items():
    x=[]
    for lista_frase in v:
        soma=0
        for palavra in lista_frase:
            try:
                soma = soma + modelo_embedding.wv[palavra]
            except:
                soma = soma
        x.append(soma)
    dicio[k] = x

In [331]:
def vetor_frases_w2v(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.wv[palavra]
                except:
                    soma = soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [332]:
def vetor_frases_ft(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.get_word_vector(palavra)
                except:
                    soma=soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [333]:
v_w2v_cbow_abstract   = vetor_frases_w2v(abstract_palavras,model_w2v_cbow)
v_w2v_cbow_introducao = vetor_frases_w2v(introducao_palavras,model_w2v_cbow)
v_w2v_cbow_conclusao  = vetor_frases_w2v(conclusao_palavras,model_w2v_cbow)

In [334]:
v_w2v_sg_abstract   = vetor_frases_w2v(abstract_palavras,model_w2v_sg)
v_w2v_sg_introducao = vetor_frases_w2v(introducao_palavras,model_w2v_sg)
v_w2v_sg_conclusao  = vetor_frases_w2v(conclusao_palavras,model_w2v_sg)

In [335]:
v_ft_cbow_abstract   = vetor_frases_ft(abstract_palavras,model_ft_cbow)
v_ft_cbow_introducao = vetor_frases_ft(introducao_palavras,model_ft_cbow)
v_ft_cbow_conclusao  = vetor_frases_ft(conclusao_palavras,model_ft_cbow)

In [336]:
v_ft_sg_abstract   = vetor_frases_ft(abstract_palavras,model_ft_sg)
v_ft_sg_introducao = vetor_frases_ft(introducao_palavras,model_ft_sg)
v_ft_sg_conclusao  = vetor_frases_ft(conclusao_palavras,model_ft_sg)

In [337]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [338]:
def aplicando_dicionario_similaridade(v1,v2):
    """cálculo da similaridade, pegando os 3 vetores mais similares"""
    x = []
    for i,j in zip(range(len(v1)),range(len(v2))):
        x.append(dicionario_similaridade(v1[i], v2[j]))
        
    top_3_indices = []
    for i in range(len(x)):
        top_3_indices.append(sorted(x[i], key=x[i].get, reverse=True)[:3])
        
    lista_1=[]
    for i in top_3_indices:
        top3=[]
        for j in i:
            top3.append(j[0])
        lista_1.append(list(set(top3)))
        
    return x,top_3_indices,lista_1

In [339]:
def frases_interseccao(lista_1,lista_2):
    """comparando se os índices similares pertencem às duas listas (abstract e conclusão) e (abstract e intro) 
    e retorna os índices em comum"""
    indices_objetivo=[]
    for i,j in zip(lista_1,lista_2):
        indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))
    return indices_objetivo

In [340]:
def get_frases_objetivo(indices_objetivo, idx_paper):
    """retorna a frase objetivo [do abstract] a partir dos indices"""
    frase_objetivo={}
    for idx,idx_f,idx_ind in zip(idx_paper,range(len(list(abstract_frases.values()))),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(list(abstract_frases.values())[idx_f][j])
            #print(idx_f)
            #print(j)
            #print(frases_abstract[idx_f][j])
        frase_objetivo[idx] = lista_one
    return frase_objetivo

### Similaridade w2v cbow

In [341]:
#Abtract e Introdução
sim_ab_intro_cbow,indices_ab_intro_cbow,lista_idx_ab_intro_cbow =aplicando_dicionario_similaridade(list(v_w2v_cbow_abstract.values()),list(v_w2v_cbow_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_cbow,indices_ab_conc_cbow,lista_idx_ab_c_cbow = aplicando_dicionario_similaridade(list(v_w2v_cbow_abstract.values()),list(v_w2v_cbow_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_cbow = frases_interseccao(lista_idx_ab_intro_cbow,lista_idx_ab_c_cbow)

#Retorna as Frases correspondentes
frase_objetivo_cbow_w2v = get_frases_objetivo(indices_objetivo_cbow, papers_treino.keys())

joblib.dump(frase_objetivo_cbow_w2v,'frase_objetivo_w2v_cbow')

### Similaridade w2v skip-gram

In [342]:
#Abtract e Introdução
sim_ab_intro_sg,indices_ab_intro_sg,lista_idx_ab_intro_sg =aplicando_dicionario_similaridade(list(v_w2v_sg_abstract.values()),list(v_w2v_sg_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_sg,indices_ab_conc_sg,lista_idx_ab_c_sg = aplicando_dicionario_similaridade(list(v_w2v_sg_abstract.values()),list(v_w2v_sg_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_sg = frases_interseccao(lista_idx_ab_intro_sg,lista_idx_ab_c_sg)

#Retorna as Frases correspondentes
frase_objetivo_sg_w2v = get_frases_objetivo(indices_objetivo_sg, papers_treino.keys())

joblib.dump(frase_objetivo_sg_w2v,'frase_objetivo_w2v_sg')

vai até 4 a 14, pois o abstract tem 5 frases e a conclusao tem 15

### Similaridade fasttext cbow

In [343]:
#Abtract e Introdução
sim_ab_intro_cbow_ft,indices_ab_intro_cbow_ft,lista_idx_ab_intro_cbow_ft =aplicando_dicionario_similaridade(list(v_ft_cbow_abstract.values()),list(v_ft_cbow_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_cbow_ft,indices_ab_conc_cbow_ft,lista_idx_ab_c_cbow_ft = aplicando_dicionario_similaridade(list(v_ft_cbow_abstract.values()),list(v_ft_cbow_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_cbow_ft = frases_interseccao(lista_idx_ab_intro_cbow_ft,lista_idx_ab_c_cbow_ft)

#Retorna as Frases correspondentes
frase_objetivo_cbow_ft  = get_frases_objetivo(indices_objetivo_cbow_ft, papers_treino.keys())

joblib.dump(frase_objetivo_cbow_ft,'frase_objetivo_cbow_ft')

In [344]:
frase_objetivo_cbow_ft

{0: ['contrary to claims found elsewhere in the literature we argue that a tagger based on markov models performs at least as well as other current approaches including the maximum entropy framework'],
 2: ['intersentence similarity is replaced by rank in the local context'],
 3: ['this paper presents a corpusbased approach to word sense disambiguation that builds an ensemble of naive bayesian classifiers each of which is based on lexical features that represent cooccurring words in varying sized windows of context'],
 4: ['we present a new parser for parsing down to penn treebank style parse trees that achieves average precisionrecall for sentences of and less and for of length and less when trained and tested on the previously established standard sections of the wall street journal treebank',
  'this represents a decrease in error rate over the best singleparser results on this corpus'],
 5: ['the system was developed and tested using essaylength responses to prompts on the test of 

In [345]:
pd.DataFrame.from_dict(frase_objetivo_cbow_ft.items()).to_excel('frase_obj.xlsx', index=False)

### Similaridade fasttext skip-gram 

In [346]:
#Abtract e Introdução
sim_ab_intro_sg_ft,indices_ab_intro_sg_ft,lista_idx_ab_intro_sg_ft =aplicando_dicionario_similaridade(list(v_ft_sg_abstract.values()),list(v_ft_sg_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_sg_ft,indices_ab_conc_sg_ft,lista_idx_ab_c_sg_ft = aplicando_dicionario_similaridade(list(v_ft_sg_abstract.values()),list(v_ft_sg_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_sg_ft = frases_interseccao(lista_idx_ab_intro_sg_ft,lista_idx_ab_c_sg_ft)

#Retorna as Frases correspondentes
frase_objetivo_sg_ft = get_frases_objetivo(indices_objetivo_sg_ft, papers_treino.keys())

In [347]:
frase_objetivo_sg_ft.keys()

dict_keys([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19, 20, 31, 32, 33, 35, 37, 39, 41, 43, 45, 47, 48, 50, 51, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 74, 88, 97, 99, 101, 102, 103, 104, 105, 106, 108, 110, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 198, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 222, 223, 225, 226, 228, 229, 231, 232, 234, 235, 243, 244, 245, 246, 248, 258, 263, 272, 276, 277, 279, 280, 281, 282, 283, 284, 286, 296, 307, 310, 313, 315, 316, 318, 323, 326, 338, 339, 342, 345, 347, 348, 350, 351, 352, 355, 356, 357, 358, 359, 362, 363, 364, 365, 366, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382

joblib.dump(frase_objetivo_sg_ft,'frase_objetivo_sg_ft')

As frases que extrai foram da variavel `abstract_frases`, ou seja, são as frases do abstract já pré-processadas

## Validação - Exp1 - ROUGE-N

### Pré-Processamento Ouro

In [348]:
def separa_frases_ouro(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('\n')
    return txt

padrao_ouro_treino_frases = {k: separa_frases_ouro(v) for k, v in padrao_ouro_treino2.items()}
padrao_ouro_treino_frases = segundo_preprocessamento(padrao_ouro_treino_frases)
padrao_ouro_treino_palavras = {k:[word_tokenize(f) for f in v] for k,v in padrao_ouro_treino_frases.items()}

### Rouge

In [349]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [350]:
def get_max(dic, coord, val):
    return max(filter(lambda item: item[0][coord] == val, dic.items()), key=lambda x: x[1])

In [351]:
def max_rouge(rouge_total):
    """pega o máximo rouge score para cada frase selecionada como pertencente ao objetivo, em comparação com cada frase
    do padrão ouro"""
    maximo_rouge_por_paper =[]
    for h in rouge_total:
        x=[]
        for i in set([i[0] for i in h]):
            x.append(get_max(h,0,i))
        maximo_rouge_por_paper.append(x)
    return maximo_rouge_por_paper

In [352]:
def calculo_rouge(frase_objetivo,lista_frases_padrao_ouro):
    
    rouge_precisao=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_1={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_1[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].precision)
        rouge_precisao.append(rouge_1)
    
    
    rouge_recall=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_2={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_2[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].recall)
        rouge_recall.append(rouge_2)
     
    
    rouge_fmeasure=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_3={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_3[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].fmeasure)
        rouge_fmeasure.append(rouge_3)
        
    return rouge_precisao, rouge_recall, rouge_fmeasure

In [353]:
def media(maximo_rouge_por_paper):
    media=[]
    for i in maximo_rouge_por_paper:
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            media.append(soma/len(i))
        except:
            media.append(0)
    return media

In [354]:
def media2(x):
    return round(np.mean(x),3)

In [355]:
p_sg_w2v,r_sg_w2v,f_sg_w2v       = calculo_rouge(list(frase_objetivo_sg_w2v.values()),list(padrao_ouro_treino_frases.values()))
p_cbow_w2v,r_cbow_w2v,f_cbow_w2v = calculo_rouge(list(frase_objetivo_cbow_w2v.values()),list(padrao_ouro_treino_frases.values()))
p_sg_ft,r_sg_ft,f_sg_ft          = calculo_rouge(list(frase_objetivo_sg_ft.values()),list(padrao_ouro_treino_frases.values()))
p_cbow_ft,r_cbow_ft,f_cbow_ft    =  calculo_rouge(list(frase_objetivo_cbow_ft.values()),list(padrao_ouro_treino_frases.values()))

In [784]:
def funcao_80_e_0(precisao,recall):
    x=[]
    y=[]
    z=[]
    for i in media(max_rouge(precisao)):
        if i>=0.8:
            x.append(i)
    for i in media(max_rouge(recall)):
        if i>=0.8:
            z.append(i)
    for i in media(max_rouge(precisao)):
        if i==0:
            y.append(i)
    print('precisao acima de 80%:', round(len(x)/len(precisao)*100,0))
    print('recall acima de 80%:', round(len(z)/len(precisao)*100,0))
    print('igual a 0:', round(len(y)/len(precisao)*100,0))
    #return len(x),len(z), len(y)

In [785]:
print('sg_w2v')
print('precisão:',media2(media(max_rouge(p_sg_w2v))))
print('revocação:',media2(media(max_rouge(r_sg_w2v))))
print('f-score:',media2(media(max_rouge(f_sg_w2v))))
funcao_80_e_0(p_sg_w2v,r_sg_w2v)

sg_w2v
precisão: 0.829
revocação: 0.85
f-score: 0.834
precisao acima de 80%: 80.0
recall acima de 80%: 86.0
igual a 0: 12.0


In [786]:
print('cbow_w2v')
print('precisão:',media2(media(max_rouge(p_cbow_w2v))))
print('revocação:',media2(media(max_rouge(r_cbow_w2v))))
print('f-score:',media2(media(max_rouge(f_cbow_w2v))))
funcao_80_e_0(p_cbow_w2v,r_cbow_w2v)

cbow_w2v
precisão: 0.808
revocação: 0.829
f-score: 0.814
precisao acima de 80%: 78.0
recall acima de 80%: 83.0
igual a 0: 14.0


In [787]:
print('sg_ft  ')
print('precisão:',media2(media(max_rouge(p_sg_ft))))
print('revocação:',media2(media(max_rouge(r_sg_ft))))
print('f-score:',media2(media(max_rouge(f_sg_ft))))
funcao_80_e_0(p_sg_ft,r_sg_ft)

sg_ft  
precisão: 0.813
revocação: 0.835
f-score: 0.819
precisao acima de 80%: 78.0
recall acima de 80%: 83.0
igual a 0: 14.0


In [788]:
print('cbow_ft')
print('precisão:',media2(media(max_rouge(p_cbow_ft))))
print('revocação:',media2(media(max_rouge(r_cbow_ft))))
print('f-score:',media2(media(max_rouge(f_cbow_ft))))
funcao_80_e_0(p_cbow_ft,r_cbow_ft)

cbow_ft
precisão: 0.832
revocação: 0.855
f-score: 0.838
precisao acima de 80%: 79.0
recall acima de 80%: 86.0
igual a 0: 11.0


In [360]:
padrao_ouro_teste_frases   = {k: separa_frases_ouro(v) for k, v in padrao_ouro_teste2.items()}
padrao_ouro_teste_frases   = segundo_preprocessamento(padrao_ouro_teste_frases)
padrao_ouro_teste_palavras = {k:[word_tokenize(f) for f in v] for k,v in padrao_ouro_teste_frases.items()}

In [361]:
lista_padrao_ouro_treino  = list(padrao_ouro_treino_frases.values())

In [362]:
lista_padrao_ouro_teste = list(padrao_ouro_teste_frases.values())

# Experimento 1 - Conclusão
Rouge Usado foi o N=1 e a precisão e recall foram calculadas a partir do f1 score

In [468]:
def analise_maior(precisao, revocacao):
    x=[]
    qtd_zero=[]
    for i in precisao:
        if i>=0.8:
            x.append(i)
        if i==0:
            qtd_zero.append(i)
                       
    y=[]
    qtd_zero=[]
    for i in revocacao:
        if i>=0.8:
            y.append(i)
            
    print('qtd de papers com precisão>=0.8:',len(x))
    print('qtd de papers com revocação>=0.8:',len(y))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    return len(x), len(y), len(qtd_zero)

def analise_revocacao(revocacao,lista_frases_padrao_ouro,media_ouro):
    x=[]
    qtd_zero=[]
    for i in revocacao:
        if i>=(1/media_ouro): #valor de 1/média qtd frases padrao ouro
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com revocação>=(1/media_ouro):',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    print('media recall', np.mean(revocacao))
    return len(x), np.mean(revocacao)

In [363]:
#Média de palavras do Padrão Ouro
x=[]
for i in lista_padrao_ouro_treino:
    x.append(len(i))
media_ouro=np.mean(x)
media_ouro

7.642424242424243

In [364]:
#Média de palavras do Padrão Ouro
x=[]
for i in lista_padrao_ouro_teste:
    x.append(len(i))
media_ouro_teste=np.mean(x)
media_ouro_teste

6.783018867924528

## Análise

import collections

def funcao_ngramas(lista_frases, n):
    todos_papers=[]
    for lista_frases_paper in lista_frases:
        perp=[]
        x=[]
        for line in lista_frases_paper:
            token = word_tokenize(line)
            ngrama = list(ngrams(token, n))
            x.append(ngrama)
    return x

bigramas = funcao_ngramas(list(frase_objetivo_cbow_ft.values()),5)

x=[]
for i in bigramas:
    for j in i:
        x.append(j)
        
counter = collections.Counter(x)

In [365]:
todos_papers=[]
for lista_frases_paper in list(frase_objetivo_cbow_ft.values()):
    for line in lista_frases_paper:
        token = word_tokenize(line)
        #print(token)
        ngrama = list(ngrams(token, 5))
        #print(ngrama)
        todos_papers.append(ngrama)

x=[]
for i in todos_papers:
    for j in i:
        x.append(j)  
        
counter = collections.Counter(x)

In [366]:
counter.most_common(20)

[(('in', 'this', 'paper', 'we', 'present'), 14),
 (('this', 'paper', 'we', 'present', 'a'), 11),
 (('in', 'this', 'paper', 'we', 'propose'), 5),
 (('this', 'paper', 'we', 'present', 'an'), 3),
 (('we', 'address', 'the', 'problem', 'of'), 3),
 (('the', 'current', 'state', 'of', 'the'), 3),
 (('paper', 'we', 'present', 'a', 'novel'), 3),
 (('an', 'error', 'reduction', 'of', 'over'), 3),
 (('performs', 'at', 'least', 'as', 'well'), 2),
 (('at', 'least', 'as', 'well', 'as'), 2),
 (('this', 'paper', 'presents', 'a', 'corpusbased'), 2),
 (('paper', 'presents', 'a', 'corpusbased', 'approach'), 2),
 (('presents', 'a', 'corpusbased', 'approach', 'to'), 2),
 (('a', 'corpusbased', 'approach', 'to', 'word'), 2),
 (('corpusbased', 'approach', 'to', 'word', 'sense'), 2),
 (('approach', 'to', 'word', 'sense', 'disambiguation'), 2),
 (('the', 'wall', 'street', 'journal', 'treebank'), 2),
 (('we', 'propose', 'a', 'method', 'for'), 2),
 (('we', 'present', 'a', 'method', 'that'), 2),
 (('present', 'a', '

In [367]:
todos_papers=[]
for lista_frases_paper in list(frase_objetivo_cbow_ft.values()):
    for line in lista_frases_paper:
        token = word_tokenize(line)
        #print(token)
        ngrama = list(ngrams(token, 4))
        #print(ngrama)
        todos_papers.append(ngrama)

x=[]
for i in todos_papers:
    for j in i:
        x.append(j)  
        
counter = collections.Counter(x)

In [368]:
counter.most_common(20)

[(('in', 'this', 'paper', 'we'), 44),
 (('this', 'paper', 'we', 'present'), 15),
 (('this', 'paper', 'presents', 'a'), 14),
 (('paper', 'we', 'present', 'a'), 11),
 (('can', 'be', 'used', 'to'), 7),
 (('we', 'present', 'a', 'novel'), 6),
 (('this', 'paper', 'describes', 'a'), 6),
 (('and', 'show', 'that', 'our'), 5),
 (('this', 'paper', 'we', 'propose'), 5),
 (('we', 'present', 'a', 'new'), 4),
 (('we', 'present', 'a', 'method'), 4),
 (('we', 'introduce', 'a', 'new'), 4),
 (('on', 'the', 'basis', 'of'), 4),
 (('state', 'of', 'the', 'art'), 4),
 (('an', 'error', 'reduction', 'of'), 4),
 (('the', 'wall', 'street', 'journal'), 3),
 (('paper', 'we', 'present', 'an'), 3),
 (('we', 'present', 'an', 'approach'), 3),
 (('we', 'address', 'the', 'problem'), 3),
 (('address', 'the', 'problem', 'of'), 3)]

# Experimento 2

`frases_objetivo_modelo` depende do modelo selecionado

https://www.cs.brandeis.edu/~cs136a/CS136a_Slides/CS136a_Lect11_PerplexityAndSmoothing.pdf

In [369]:
def funcao_start_end_sentences(frase_objetivo_ft_formato_ml):
    lista=[]
    for i in frase_objetivo_ft_formato_ml:
        x = ['<s>'] + i + ['</s>']
        lista.append(x)
    return lista

def funcao_aplicar_es_sentencer(dicionario):
    """Essa função aplica o <s> e o </s> em cada frase
    entrada: dicionario de palavras (ex: palavras_papers_teste ou palavras_papers_treino)
    saida: lista com os tokens de palavras de frases com o start e end of sentences"""
    x=[]
    for i in range(len(dicionario.values())):
        x.append(funcao_start_end_sentences(list(dicionario.values())[i]))
    return x

frases_objetivo_modelo = frase_objetivo_cbow_ft.copy()
palavras_objetivo_modelo = {k:[word_tokenize(f) for f in v] for k,v in frases_objetivo_modelo.items()}

#O N-gramas é calculado a partir de uma lista com listas de palavras de cada frase
frase_objetivo_ft_formato_ml=[]
for i in list(palavras_objetivo_modelo.values()):
    for j in i:
        frase_objetivo_ft_formato_ml.append(j)
frase_objetivo_ft_formato_ml = funcao_start_end_sentences(frase_objetivo_ft_formato_ml)

frases_objetivo_modelo = frase_objetivo_cbow_ft.copy()
palavras_objetivo_modelo = {k:[word_tokenize(f) for f in v] for k,v in frases_objetivo_modelo.items()}

#O N-gramas é calculado a partir de uma lista com listas de palavras de cada frase
frase_objetivo_ft_formato_ml=[]
for i in list(palavras_objetivo_modelo.values()):
    for j in i:
        frase_objetivo_ft_formato_ml.append(j)
frase_objetivo_ft_formato_ml = (frase_objetivo_ft_formato_ml)

In [370]:
frases_objetivo_modelo = frase_objetivo_cbow_ft.copy()

x=[]
for i in list(frase_objetivo_cbow_ft.values()):
    for j in i:
        x.append(j)
frase_objetivo_ft_formato_ml = x.copy()

In [371]:
def funcao_perplexidade(lista_frases, n, modelo):
    todos_papers=[]
    for lista_frases_paper in lista_frases:
        perp=[]
        x=[]
        for line in lista_frases_paper:
            token = word_tokenize(line)
            ngrama = list(ngrams(token, n, pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>"))
            x.append(ngrama)

        for bigramas_frase in x:
            #print(model_laplace_bi.perplexity(bigramas_frase))
            perp.append(modelo.perplexity(bigramas_frase))
        todos_papers.append(perp)
    return todos_papers

In [372]:
def media_perplexidade(lista_perplexidade_todos_papers):
    media = [np.mean(i) for i in lista_perplexidade_todos_papers]
    return (np.mean(media))

In [373]:
def lista_sem_infinito(lista_perplexidade):
    lista_sem_infinito=[]
    for i in range(len(lista_perplexidade)):
        x = [v for v in lista_perplexidade[i] if not np.isnan(v) and not np.isinf(v)]
        lista_sem_infinito.append(x)
    return lista_sem_infinito

def media_lista_sem_infinito(input_lista_sem_infinito):
    x=[]
    for i in input_lista_sem_infinito:
        x.append(np.mean(i))
    x = [v for v in x if not np.isnan(v) and not np.isinf(v)]
    return np.mean(x)

In [374]:
def token_frases(lista_frases):
    """lista_frases = frases_papers_treino ou frases_papers_teste"""
    x=[]
    for j in lista_frases:
        h=[]
        for z in j:
            h.append(nltk.tokenize.word_tokenize(z))
        x.append(h)
    return x


def gerador_ngramas(token_frases, n):
    x=[]
    for i in token_frases:
        z=[]
        for j in i:
            #print(list(nltk.ngrams(j, n, pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>")))
            z.append(list(nltk.ngrams(j, n, pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>")))
        x.append(z)
    return x

def gerador_probabilidades(ngramas, modelo):
    """exemplo:
    ngramas=bigramas_treino
    modelo = model_bi"""
    geral=[]
    for a in ngramas:
        x=[]
        for b in a:
            prob=1
            for i in b:
                #print(i, model_bi.score(i[-1], i[:-1]))
                prob = prob * modelo.score(i[-1], i[:-1])
            #print(prob)
            x.append(prob)
        geral.append(x)
    return geral

In [375]:
def selecao_frases_obj_mle(probabibilidade, frases_papers):
    """função para selecionar até 3 maiores elementos da lista"""
    #Pega os 3 elementos de maior probabilidade
    lista=[]
    dicionario={}
    x=[]
    for i,j in zip(range(len(probabibilidade)),range(len(frases_papers))):
        lista.append(sorted(zip(probabibilidade[i],frases_papers[j]), reverse=True)[:3])
    return lista

In [376]:
def filtro_selecao_frases_mle(frase_objetivo_trigramas_treino):
    lista_frase=[]
    lista_prob=[]
    lista_geral=[]
    for i in frase_objetivo_trigramas_treino:
        frase=[]
        prob=[]
        geral=[]
        for j,h in i:
            if j!=0:
                frase.append(h)
                prob.append(j)
                geral.append((h,j))
        lista_frase.append(frase)
        lista_prob.append(prob)
        lista_geral.append(geral)
    return lista_frase,lista_prob,lista_geral

In [377]:
frases_papers_treino2 = list(frases_papers_treino.values())
frases_papers_teste2 = list(frases_papers_teste.values())

In [378]:
token_frases_treino     = token_frases(frases_papers_treino2)

In [379]:
token_frases_teste     = token_frases(frases_papers_teste2)

## Unigrama

### MLE

In [380]:
if os.path.exists('model_uni'):
    model_uni = joblib.load('model_uni')
else:
    n = 1
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_uni = MLE(n)
    model_uni.fit(train_data, padded_sents)

    print(model_uni.vocab)

n = 1
paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

model_uni = MLE(n)
model_uni.fit(train_data, padded_sents)

print(model_uni.vocab)

joblib.dump(model_uni,'model_uni')

#### Perplexidade Treino

In [381]:
unigramas_treino          = gerador_ngramas(token_frases_treino,1)
probabilidades_unigramas  = gerador_probabilidades(unigramas_treino, model_uni)

unigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 1 , model_uni)
media_unigramas_perp_treino = media_perplexidade(unigramas_perp_treino)

unigramas_perp_treino_s_inf = lista_sem_infinito(unigramas_perp_treino)
media_unigramas_perp_treino_s_inf = media_lista_sem_infinito(unigramas_perp_treino_s_inf)

print('média:',media_unigramas_perp_treino)
print('média sem infinito:', media_unigramas_perp_treino_s_inf)

média: inf
média sem infinito: 994.6652048610426


#### Rouge Treino

In [382]:
frase_objetivo_unigramas_treino         = selecao_frases_obj_mle(probabilidades_unigramas, frases_papers_treino2)
lista_frase_uni,lista_prob_uni,lista_geral_uni      = filtro_selecao_frases_mle(frase_objetivo_unigramas_treino)

In [383]:
p_uni_treino,r_uni_treino,f_uni_treino  = calculo_rouge(lista_frase_uni,list(padrao_ouro_treino_frases.values()))

In [789]:
print('unigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_uni_treino))))
print('revocação:',media2(media(max_rouge(r_uni_treino))))
print('f-score:',media2(media(max_rouge(f_uni_treino))))
funcao_80_e_0(p_uni_treino,r_uni_treino)

unigrama treino sem laplace
precisão: 0.447
revocação: 0.223
f-score: 0.254
precisao acima de 80%: 5.0
recall acima de 80%: 1.0
igual a 0: 4.0


rouge_unigramas                         = calculo_rouge(lista_frase_uni,list(padrao_ouro_treino_frases.values()))
maximo_rouge_unigramas                  = max_rouge(rouge_unigramas) #lista de índices e maior rouge
precisao_unigramas, revocacao_unigramas = rouge_precisao_revocacao(maximo_rouge_unigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_unigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_unigramas, lista_padrao_ouro_treino)

#### Perplexidade Teste

In [385]:
unigramas_teste          = gerador_ngramas(token_frases_teste,1)
probabilidades_unigramas  = gerador_probabilidades(unigramas_teste, model_uni)

unigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 1 , model_uni)
media_unigramas_perp_teste = media_perplexidade(unigramas_perp_teste)

unigramas_perp_teste_s_inf = lista_sem_infinito(unigramas_perp_teste)
media_unigramas_perp_teste_s_inf = media_lista_sem_infinito(unigramas_perp_teste_s_inf)

print('média:',media_unigramas_perp_teste)
print('média sem infinito:', media_unigramas_perp_teste_s_inf)

média: inf
média sem infinito: 998.6757506011328


#### Rouge Teste

In [386]:
frase_objetivo_unigramas_teste                     = selecao_frases_obj_mle(probabilidades_unigramas, frases_papers_teste2)
lista_frase_uni,lista_prob_uni,lista_geral_uni     = filtro_selecao_frases_mle(frase_objetivo_unigramas_teste)

In [387]:
p_uni_teste,r_uni_teste,f_uni_teste  = calculo_rouge(lista_frase_uni,list(padrao_ouro_teste_frases.values()))

In [790]:
print('unigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_uni_teste))))
print('revocação:',media2(media(max_rouge(r_uni_teste))))
print('f-score:',media2(media(max_rouge(f_uni_teste))))
funcao_80_e_0(p_uni_teste,r_uni_teste)

unigrama teste sem laplace
precisão: 0.419
revocação: 0.23
f-score: 0.258
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 4.0


rouge_unigramas                         = calculo_rouge(lista_frase_uni,list(padrao_ouro_teste_frases.values()))
maximo_rouge_unigramas                  = max_rouge(rouge_unigramas) #lista de índices e maior rouge
precisao_unigramas, revocacao_unigramas = rouge_precisao_revocacao(maximo_rouge_unigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_unigramas, lista_padrao_ouro_teste, media_ouro_teste)
print('----')
analise_precisao(precisao_unigramas, lista_padrao_ouro_teste)

### Laplace

In [389]:
if os.path.exists('model_uni_laplace'):
    model_uni_laplace = joblib.load('model_uni_laplace')
else:
    n = 1
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_uni_laplace = Laplace(n)
    model_uni_laplace.fit(train_data, padded_sents)

#### Treino

In [390]:
unigramas_treino          = gerador_ngramas(token_frases_treino,1)
probabilidades_unigramas_laplace  = gerador_probabilidades(unigramas_treino, model_uni_laplace)

unigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 1 , model_uni_laplace)
media_unigramas_perp_treino = media_perplexidade(unigramas_perp_treino)

unigramas_perp_treino_s_inf = lista_sem_infinito(unigramas_perp_treino)
media_unigramas_perp_treino_s_inf = media_lista_sem_infinito(unigramas_perp_treino_s_inf)

print('média:',media_unigramas_perp_treino)
print('média sem infinito:', media_unigramas_perp_treino_s_inf)

frase_objetivo_unigramas_treino         = selecao_frases_obj_mle(probabilidades_unigramas_laplace, frases_papers_treino2)
lista_frase_uni,lista_prob_uni,lista_geral_uni      = filtro_selecao_frases_mle(frase_objetivo_unigramas_treino)

média: 1329.0739210268796
média sem infinito: 1329.0739210268796


In [391]:
p_uni_treino_laplace,r_uni_treino_laplace,f_uni_treino_laplace  = calculo_rouge(lista_frase_uni,list(padrao_ouro_treino_frases.values()))

In [791]:
print('unigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_uni_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_uni_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_uni_treino_laplace))))
funcao_80_e_0(p_uni_treino_laplace,r_uni_treino_laplace)

unigrama treino com laplace
precisão: 0.347
revocação: 0.138
f-score: 0.167
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 11.0



rouge_unigramas                         = calculo_rouge(lista_frase_uni,list(padrao_ouro_treino_frases.values()))
maximo_rouge_unigramas                  = max_rouge(rouge_unigramas) #lista de índices e maior rouge
precisao_unigramas, revocacao_unigramas = rouge_precisao_revocacao(maximo_rouge_unigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_unigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_unigramas, lista_padrao_ouro_treino)

#### Teste

In [393]:
unigramas_teste          = gerador_ngramas(token_frases_teste,1)
probabilidades_unigramas_laplace_teste  = gerador_probabilidades(unigramas_teste, model_uni_laplace)

unigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 1 , model_uni_laplace)
media_unigramas_perp_teste = media_perplexidade(unigramas_perp_teste)

unigramas_perp_teste_s_inf = lista_sem_infinito(unigramas_perp_teste)
media_unigramas_perp_teste_s_inf = media_lista_sem_infinito(unigramas_perp_teste_s_inf)

print('média:',media_unigramas_perp_teste)
print('média sem infinito:', media_unigramas_perp_teste_s_inf)

frase_objetivo_unigramas_teste         = selecao_frases_obj_mle(probabilidades_unigramas_laplace_teste, frases_papers_teste2)
lista_frase_uni,lista_prob_uni,lista_geral_uni      = filtro_selecao_frases_mle(frase_objetivo_unigramas_teste)

média: 1294.3986490998302
média sem infinito: 1294.3986490998302


In [394]:
p_uni_teste_laplace,r_uni_teste_laplace,f_uni_teste_laplace  = calculo_rouge(lista_frase_uni,list(padrao_ouro_teste_frases.values()))

In [792]:
print('unigrama teste com laplace')
print('precisão:',media2(media(max_rouge(p_uni_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_uni_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_uni_teste_laplace))))
funcao_80_e_0(p_uni_teste_laplace,r_uni_teste_laplace)

unigrama teste com laplace
precisão: 0.351
revocação: 0.164
f-score: 0.191
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 9.0


rouge_unigramas                         = calculo_rouge(lista_frase_uni,list(padrao_ouro_teste_frases.values()))
maximo_rouge_unigramas                  = max_rouge(rouge_unigramas) #lista de índices e maior rouge
precisao_unigramas, revocacao_unigramas = rouge_precisao_revocacao(maximo_rouge_unigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_unigramas, lista_padrao_ouro_teste, media_ouro_teste)
print('----')
analise_precisao(precisao_unigramas, lista_padrao_ouro_teste)

## Bigrama

In [396]:
if os.path.exists('model_bi'):
    model_bi = joblib.load('model_bi')
else:
    n = 2
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_bi = MLE(n)
    model_bi.fit(train_data, padded_sents)

n = 2
paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

model_bi = MLE(n)
model_bi.fit(train_data, padded_sents)

print(model_bi.vocab)

joblib.dump(model_bi,'model_bi')

### MLE

#### Perplexidade Treino

In [397]:
bigramas_treino          = gerador_ngramas(token_frases_treino,2)
probabilidades_bigramas  = gerador_probabilidades(bigramas_treino, model_bi)

bigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 2 , model_bi)
media_bigramas_perp_treino = media_perplexidade(bigramas_perp_treino)

bigramas_perp_treino_s_inf       = lista_sem_infinito(bigramas_perp_treino)
media_bigramas_perp_treino_s_inf = media_lista_sem_infinito(bigramas_perp_treino_s_inf)


print('média:',media_bigramas_perp_treino)
print('média sem infinito:', media_bigramas_perp_treino_s_inf)

média: inf
média sem infinito: 20.823422762126768


#### Rouge Treino

In [398]:
frase_objetivo_bigramas_treino    = selecao_frases_obj_mle(probabilidades_bigramas, frases_papers_treino2)
lista_frase_bigramas,lista_prob_bigramas,lista_geral_bigramas = filtro_selecao_frases_mle(frase_objetivo_bigramas_treino)

In [399]:
p_bi_treino,r_bi_treino,f_bi_treino     = calculo_rouge(lista_frase_bigramas,list(padrao_ouro_treino_frases.values()))

In [793]:
print('bigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_bi_treino))))
print('revocação:',media2(media(max_rouge(r_bi_treino))))
print('f-score:',media2(media(max_rouge(f_bi_treino))))
funcao_80_e_0(p_bi_treino,r_bi_treino)

bigrama treino sem laplace
precisão: 0.827
revocação: 0.817
f-score: 0.805
precisao acima de 80%: 76.0
recall acima de 80%: 77.0
igual a 0: 10.0


rouge_bigramas                         = calculo_rouge(lista_frase_bigramas,list(padrao_ouro_treino_frases.values()))
maximo_rouge_bigramas                  = max_rouge(rouge_bigramas) #lista de índices e maior rouge
precisao_bigramas, revocacao_bigramas = rouge_precisao_revocacao(maximo_rouge_bigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_bigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_bigramas, lista_padrao_ouro_treino)

#### Perplexidade Teste

In [401]:
bigramas_teste          = gerador_ngramas(token_frases_teste,2)
probabilidades_bigramas  = gerador_probabilidades(bigramas_teste, model_bi)

bigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 2 , model_bi)
media_bigramas_perp_teste = media_perplexidade(bigramas_perp_teste)

bigramas_perp_teste_s_inf       = lista_sem_infinito(bigramas_perp_teste)
media_bigramas_perp_teste_s_inf = media_lista_sem_infinito(bigramas_perp_teste_s_inf)

In [402]:
print('média:',media_bigramas_perp_teste)
print('média sem infinito:', media_bigramas_perp_teste_s_inf)

média: inf
média sem infinito: 104.49078017502863


#### Rouge Teste

In [403]:
frase_objetivo_bigramas_teste    = selecao_frases_obj_mle(probabilidades_bigramas, frases_papers_teste2)
lista_frase_teste_bigramas,lista_prob_teste_bigramas,lista_geral_teste_bigramas = filtro_selecao_frases_mle(frase_objetivo_bigramas_teste)

In [404]:
p_bi_teste,r_bi_teste,f_bi_teste  = calculo_rouge(lista_frase_teste_bigramas,list(padrao_ouro_teste_frases.values()))

In [794]:
print('bigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_bi_teste))))
print('revocação:',media2(media(max_rouge(r_bi_teste))))
print('f-score:',media2(media(max_rouge(f_bi_teste))))
funcao_80_e_0(p_bi_teste,r_bi_teste)

bigrama teste sem laplace
precisão: 0.042
revocação: 0.013
f-score: 0.015
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 95.0


rouge_bigramas                         = calculo_rouge(lista_frase_teste_bigramas,list(padrao_ouro_teste_frases.values()))
maximo_rouge_bigramas                  = max_rouge(rouge_bigramas) #lista de índices e maior rouge
precisao_bigramas, revocacao_bigramas = rouge_precisao_revocacao(maximo_rouge_bigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_bigramas, lista_padrao_ouro_teste, media_ouro_teste)
print('----')
analise_precisao(precisao_bigramas, lista_padrao_ouro_teste)

### Laplace

In [406]:
if os.path.exists('model_bi_laplace'):
    model_bi_laplace = joblib.load('model_bi_laplace')
else:
    n = 2
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_bi_laplace = Laplace(n)
    model_bi_laplace.fit(train_data, padded_sents)

#### Treino

In [407]:
bigramas_treino          = gerador_ngramas(token_frases_treino,2)
probabilidades_bigramas_laplace  = gerador_probabilidades(bigramas_treino, model_bi_laplace)

bigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 2 , model_bi_laplace)
media_bigramas_perp_treino = media_perplexidade(bigramas_perp_treino)

bigramas_perp_treino_s_inf       = lista_sem_infinito(bigramas_perp_treino)
media_bigramas_perp_treino_s_inf = media_lista_sem_infinito(bigramas_perp_treino_s_inf)


print('média:',media_bigramas_perp_treino)
print('média sem infinito:', media_bigramas_perp_treino_s_inf)

média: 1835.620010581891
média sem infinito: 1835.620010581891


In [408]:
frase_objetivo_bigramas_treino    = selecao_frases_obj_mle(probabilidades_bigramas_laplace, frases_papers_treino2)
lista_frase_bigramas,lista_prob_bigramas,lista_geral_bigramas = filtro_selecao_frases_mle(frase_objetivo_bigramas_treino)

In [409]:
p_bi_treino_laplace,r_bi_treino_laplace,f_bi_treino_laplace  = calculo_rouge(lista_frase_bigramas,list(padrao_ouro_treino_frases.values()))

In [795]:
print('bigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_bi_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_bi_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_bi_treino_laplace))))
funcao_80_e_0(p_bi_treino_laplace,r_bi_treino_laplace)

bigrama treino com laplace
precisão: 0.323
revocação: 0.134
f-score: 0.161
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 15.0


rouge_bigramas                         = calculo_rouge(lista_frase_bigramas,list(padrao_ouro_treino_frases.values()))
maximo_rouge_bigramas                  = max_rouge(rouge_bigramas) #lista de índices e maior rouge
precisao_bigramas, revocacao_bigramas = rouge_precisao_revocacao(maximo_rouge_bigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_bigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_bigramas, lista_padrao_ouro_treino)

#### Teste

In [411]:
bigramas_teste          = gerador_ngramas(token_frases_teste,2)
probabilidades_bigramas_laplace  = gerador_probabilidades(bigramas_teste, model_bi_laplace)

bigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 2 , model_bi_laplace)
media_bigramas_perp_teste = media_perplexidade(bigramas_perp_teste)

bigramas_perp_teste_s_inf       = lista_sem_infinito(bigramas_perp_teste)
media_bigramas_perp_teste_s_inf = media_lista_sem_infinito(bigramas_perp_teste_s_inf)

In [412]:
print('media_bigramas_perp_teste',media_bigramas_perp_teste)
print('media_bigramas_perp_teste_s_inf',media_bigramas_perp_teste_s_inf)

media_bigramas_perp_teste 1851.2007455554535
media_bigramas_perp_teste_s_inf 1851.2007455554535


In [413]:
frase_objetivo_bigramas_teste    = selecao_frases_obj_mle(probabilidades_bigramas_laplace, frases_papers_teste2)
lista_frase_teste_bigramas,lista_prob_teste_bigramas,lista_geral_teste_bigramas = filtro_selecao_frases_mle(frase_objetivo_bigramas_teste)

In [414]:
p_bi_teste_laplace,r_bi_teste_laplace,f_bi_teste_laplace  = calculo_rouge(lista_frase_teste_bigramas,list(padrao_ouro_teste_frases.values()))

In [796]:
print('bigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_bi_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_bi_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_bi_teste_laplace))))
funcao_80_e_0(p_bi_teste_laplace,r_bi_teste_laplace)

bigrama teste sem laplace
precisão: 0.325
revocação: 0.153
f-score: 0.179
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 12.0


rouge_bigramas                         = calculo_rouge(lista_frase_teste_bigramas,list(padrao_ouro_teste_frases.values()))
maximo_rouge_bigramas                  = max_rouge(rouge_bigramas) #lista de índices e maior rouge
precisao_bigramas, revocacao_bigramas = rouge_precisao_revocacao(maximo_rouge_bigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_bigramas, lista_padrao_ouro_teste, media_ouro_teste)
print('----')
analise_precisao(precisao_bigramas, lista_padrao_ouro_teste)

In [416]:
media_bigramas_perp_teste

1851.2007455554535

## Trigrama

In [417]:
if os.path.exists('model_tri'):
    model_tri = joblib.load('model_tri')
else:
    n = 3
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_tri = MLE(n)
    model_tri.fit(train_data, padded_sents)

    print(model_tri.vocab)

n = 3
paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

model_tri = MLE(n)
model_tri.fit(train_data, padded_sents)

print(model_tri.vocab)

joblib.dump(model_tri,'model_tri')

### MLE

#### Perplexidade Treino

In [418]:
trigramas_treino         = gerador_ngramas(token_frases_treino,3)
probabilidades_trigramas = gerador_probabilidades(trigramas_treino, model_tri)

trigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 3 , model_tri)
media_trigramas_perp_treino = media_perplexidade(trigramas_perp_treino)

trigramas_perp_treino_s_inf       = lista_sem_infinito(trigramas_perp_treino)
media_trigramas_perp_treino_s_inf = media_lista_sem_infinito(trigramas_perp_treino_s_inf)

In [419]:
print('média:',media_trigramas_perp_treino)
print('média sem infinito:', media_trigramas_perp_treino_s_inf)

média: inf
média sem infinito: 2.3478837827173082


#### Rouge Treino

In [420]:
frase_objetivo_trigramas_treino    = selecao_frases_obj_mle(probabilidades_trigramas, frases_papers_treino2)
lista_frase,lista_prob,lista_geral = filtro_selecao_frases_mle(frase_objetivo_trigramas_treino)

In [421]:
p_tri_treino,r_tri_treino,f_tri_treino    = calculo_rouge(lista_frase,list(padrao_ouro_treino_frases.values()))

In [797]:
print('trigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_tri_treino))))
print('revocação:',media2(media(max_rouge(r_tri_treino))))
print('f-score:',media2(media(max_rouge(f_tri_treino))))
funcao_80_e_0(p_tri_treino,r_tri_treino)

trigrama treino sem laplace
precisão: 0.832
revocação: 0.854
f-score: 0.838
precisao acima de 80%: 79.0
recall acima de 80%: 86.0
igual a 0: 11.0


rouge_trigramas                         = calculo_rouge(lista_frase,list(padrao_ouro_treino_frases.values()))
maximo_rouge_trigramas                  = max_rouge(rouge_trigramas) #lista de índices e maior rouge
precisao_trigramas, revocacao_trigramas = rouge_precisao_revocacao(maximo_rouge_trigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_trigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_trigramas, lista_padrao_ouro_treino)

#### Perplexidade Teste

In [423]:
trigramas_teste          = gerador_ngramas(token_frases_teste,3)
probabilidades_trigramas = gerador_probabilidades(trigramas_teste, model_tri)

trigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 3 , model_tri)
media_trigramas_perp_teste = media_perplexidade(trigramas_perp_teste)

trigramas_perp_teste_s_inf       = lista_sem_infinito(trigramas_perp_teste)
media_trigramas_perp_teste_s_inf = media_lista_sem_infinito(trigramas_perp_teste_s_inf)

In [424]:
print('média:',media_trigramas_perp_teste)
print('média sem infinito:', media_trigramas_perp_teste_s_inf)

média: inf
média sem infinito: 8.015579431135096


#### Rouge Teste

In [426]:
frase_objetivo_trigramas_teste    = selecao_frases_obj_mle(probabilidades_trigramas, frases_papers_teste2)
lista_frase_teste,lista_prob_teste,lista_geral_teste = filtro_selecao_frases_mle(frase_objetivo_trigramas_teste)

In [427]:
p_tri_teste,r_tri_teste,f_tri_teste  = calculo_rouge(lista_frase_teste,list(padrao_ouro_teste_frases.values()))

In [798]:
print('trigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_tri_teste))))
print('revocação:',media2(media(max_rouge(r_tri_teste))))
print('f-score:',media2(media(max_rouge(f_tri_teste))))
funcao_80_e_0(p_tri_teste,r_tri_teste)

trigrama teste sem laplace
precisão: 0.0
revocação: 0.0
f-score: 0.0
precisao acima de 80%: 0.0
recall acima de 80%: 0.0
igual a 0: 100.0


frase_objetivo_trigramas_teste    = selecao_frases_obj_mle(probabilidades_trigramas, frases_papers_teste2)
lista_frase_teste,lista_prob_teste,lista_geral_teste = filtro_selecao_frases_mle(frase_objetivo_trigramas_teste)

rouge_trigramas                         = calculo_rouge(lista_frase_teste,list(padrao_ouro_teste_frases.values()))
maximo_rouge_trigramas                  = max_rouge(rouge_trigramas) #lista de índices e maior rouge
precisao_trigramas, revocacao_trigramas = rouge_precisao_revocacao(maximo_rouge_trigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_trigramas, lista_padrao_ouro_teste, media_ouro)
print('----')
analise_precisao(precisao_trigramas, lista_padrao_ouro_teste)

### Laplace

In [432]:
if os.path.exists('model_tri_laplace'):
    model_tri_laplace = joblib.load('model_tri_laplace')
else:
    n = 3
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_tri_laplace = Laplace(n)
    model_tri_laplace.fit(train_data, padded_sents)

    print(model_tri_laplace.vocab)

#### Treino

In [433]:
trigramas_treino         = gerador_ngramas(token_frases_treino,3)
probabilidades_trigramas = gerador_probabilidades(trigramas_treino, model_tri_laplace)

trigramas_perp_treino       = funcao_perplexidade(list(frases_papers_treino.values()), 3 , model_tri_laplace)
media_trigramas_perp_treino = media_perplexidade(trigramas_perp_treino)

trigramas_perp_treino_s_inf       = lista_sem_infinito(trigramas_perp_treino)
media_trigramas_perp_treino_s_inf = media_lista_sem_infinito(trigramas_perp_treino_s_inf)

In [434]:
print(media_trigramas_perp_treino)

2593.6182312686974


In [435]:
frase_objetivo_trigramas_treino    = selecao_frases_obj_mle(probabilidades_trigramas, frases_papers_treino2)
lista_frase,lista_prob,lista_geral = filtro_selecao_frases_mle(frase_objetivo_trigramas_treino)

In [436]:
p_tri_treino_laplace,r_tri_treino_laplace,f_tri_treino_laplace  = calculo_rouge(lista_frase,list(padrao_ouro_treino_frases.values()))

In [799]:
print('trigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_tri_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_tri_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_tri_treino_laplace))))
funcao_80_e_0(p_tri_treino_laplace,r_tri_treino_laplace)

trigrama treino com laplace
precisão: 0.32
revocação: 0.129
f-score: 0.155
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 14.0


rouge_trigramas                         = calculo_rouge(lista_frase,list(padrao_ouro_treino_frases.values()))
maximo_rouge_trigramas                  = max_rouge(rouge_trigramas) #lista de índices e maior rouge
precisao_trigramas, revocacao_trigramas = rouge_precisao_revocacao(maximo_rouge_trigramas, lista_padrao_ouro_treino)
analise_revocacao(revocacao_trigramas, lista_padrao_ouro_treino, media_ouro)
print('----')
analise_precisao(precisao_trigramas, lista_padrao_ouro_treino)

#### Teste

In [438]:
trigramas_teste          = gerador_ngramas(token_frases_teste,3)
probabilidades_trigramas = gerador_probabilidades(trigramas_teste, model_tri_laplace)

trigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 3 , model_tri_laplace)
media_trigramas_perp_teste = media_perplexidade(trigramas_perp_teste)

trigramas_perp_teste_s_inf       = lista_sem_infinito(trigramas_perp_teste)
media_trigramas_perp_teste_s_inf = media_lista_sem_infinito(trigramas_perp_teste_s_inf)

In [439]:
print(media_trigramas_perp_teste)

2615.8237276236473


In [440]:
frase_objetivo_trigramas_teste    = selecao_frases_obj_mle(probabilidades_trigramas, frases_papers_teste2)
lista_frase_teste,lista_prob_teste,lista_geral_teste = filtro_selecao_frases_mle(frase_objetivo_trigramas_teste)


In [441]:
p_tri_teste_laplace,r_tri_teste_laplace,f_tri_teste_laplace  = calculo_rouge(lista_frase_teste,list(padrao_ouro_teste_frases.values()))

In [800]:
print('trigrama teste com laplace')
print('precisão:',media2(media(max_rouge(p_tri_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_tri_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_tri_teste_laplace))))
funcao_80_e_0(p_tri_teste_laplace,r_tri_teste_laplace)

trigrama teste com laplace
precisão: 0.33
revocação: 0.153
f-score: 0.18
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 11.0


rouge_trigramas                         = calculo_rouge(lista_frase_teste,list(padrao_ouro_teste_frases.values()))
maximo_rouge_trigramas                  = max_rouge(rouge_trigramas) #lista de índices e maior rouge
precisao_trigramas, revocacao_trigramas = rouge_precisao_revocacao(maximo_rouge_trigramas, lista_padrao_ouro_teste)
analise_revocacao(revocacao_trigramas, lista_padrao_ouro_teste, media_ouro)
print('----')
analise_precisao(precisao_trigramas, lista_padrao_ouro_teste)

In [443]:
media_trigramas_perp_teste

2615.8237276236473

## rouge geral

Exemplos de frases e suas probabilidades

In [801]:
print('TREINO SEM LAPLACE')
print('unigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_uni_treino))))
print('revocação:',media2(media(max_rouge(r_uni_treino))))
print('f-score:',media2(media(max_rouge(f_uni_treino))))
funcao_80_e_0(p_uni_treino,r_uni_treino)

print('-----')
print('bigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_bi_treino))))
print('revocação:',media2(media(max_rouge(r_bi_treino))))
print('f-score:',media2(media(max_rouge(f_bi_treino))))
funcao_80_e_0(p_bi_treino,r_bi_treino)

print('-----')
print('trigrama treino sem laplace')
print('precisão:',media2(media(max_rouge(p_tri_treino))))
print('revocação:',media2(media(max_rouge(r_tri_treino))))
print('f-score:',media2(media(max_rouge(f_tri_treino))))
funcao_80_e_0(p_tri_treino,r_tri_treino)

TREINO SEM LAPLACE
unigrama treino sem laplace
precisão: 0.447
revocação: 0.223
f-score: 0.254
precisao acima de 80%: 5.0
recall acima de 80%: 1.0
igual a 0: 4.0
-----
bigrama treino sem laplace
precisão: 0.827
revocação: 0.817
f-score: 0.805
precisao acima de 80%: 76.0
recall acima de 80%: 77.0
igual a 0: 10.0
-----
trigrama treino sem laplace
precisão: 0.832
revocação: 0.854
f-score: 0.838
precisao acima de 80%: 79.0
recall acima de 80%: 86.0
igual a 0: 11.0


In [807]:
print('TESTE SEM LAPLACE')
print('unigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_uni_teste))))
print('revocação:',media2(media(max_rouge(r_uni_teste))))
print('f-score:',media2(media(max_rouge(f_uni_teste))))
funcao_80_e_0(p_uni_teste,r_uni_teste)

print('-----')
print('bigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_bi_teste))))
print('revocação:',media2(media(max_rouge(r_bi_teste))))
print('f-score:',media2(media(max_rouge(f_bi_teste))))
funcao_80_e_0(p_bi_teste,r_bi_teste)

print('-----')
print('trigrama teste sem laplace')
print('precisão:',media2(media(max_rouge(p_tri_teste))))
print('revocação:',media2(media(max_rouge(r_tri_teste))))
print('f-score:',media2(media(max_rouge(f_tri_teste))))
funcao_80_e_0(p_tri_teste,r_tri_teste)

TESTE SEM LAPLACE
unigrama teste sem laplace
precisão: 0.419
revocação: 0.23
f-score: 0.258
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 4.0
-----
bigrama teste sem laplace
precisão: 0.042
revocação: 0.013
f-score: 0.015
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 95.0
-----
trigrama teste sem laplace
precisão: 0.0
revocação: 0.0
f-score: 0.0
precisao acima de 80%: 0.0
recall acima de 80%: 0.0
igual a 0: 100.0


In [810]:
print('TREINO COM LAPLACE')
print('unigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_uni_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_uni_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_uni_treino_laplace))))
funcao_80_e_0(p_uni_treino_laplace,r_uni_treino_laplace)
print('-----')
print('bigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_bi_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_bi_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_bi_treino_laplace))))
funcao_80_e_0(p_bi_treino_laplace,r_bi_treino_laplace)
print('-----')
print('trigrama treino com laplace')
print('precisão:',media2(media(max_rouge(p_tri_treino_laplace))))
print('revocação:',media2(media(max_rouge(r_tri_treino_laplace))))
print('f-score:',media2(media(max_rouge(f_tri_treino_laplace))))
funcao_80_e_0(p_tri_treino_laplace,r_tri_treino_laplace)

TREINO COM LAPLACE
unigrama treino com laplace
precisão: 0.347
revocação: 0.138
f-score: 0.167
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 11.0
-----
bigrama treino com laplace
precisão: 0.323
revocação: 0.134
f-score: 0.161
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 15.0
-----
trigrama treino com laplace
precisão: 0.32
revocação: 0.129
f-score: 0.155
precisao acima de 80%: 3.0
recall acima de 80%: 0.0
igual a 0: 14.0


In [811]:
print('TESTE COM LAPLACE')
print('unigrama teste com laplace')
print('precisão:',media2(media(max_rouge(p_uni_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_uni_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_uni_teste_laplace))))
funcao_80_e_0(p_uni_teste_laplace,r_uni_teste_laplace)
print('-----')
print('bigrama teste com laplace')
print('precisão:',media2(media(max_rouge(p_bi_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_bi_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_bi_teste_laplace))))
funcao_80_e_0(p_bi_teste_laplace,r_bi_teste_laplace)
print('-----')
print('trigrama teste com laplace')
print('precisão:',media2(media(max_rouge(p_tri_teste_laplace))))
print('revocação:',media2(media(max_rouge(r_tri_teste_laplace))))
print('f-score:',media2(media(max_rouge(f_tri_teste_laplace))))
funcao_80_e_0(p_tri_teste_laplace,r_tri_teste_laplace)

TESTE COM LAPLACE
unigrama teste com laplace
precisão: 0.351
revocação: 0.164
f-score: 0.191
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 9.0
-----
bigrama teste com laplace
precisão: 0.325
revocação: 0.153
f-score: 0.179
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 12.0
-----
trigrama teste com laplace
precisão: 0.33
revocação: 0.153
f-score: 0.18
precisao acima de 80%: 2.0
recall acima de 80%: 0.0
igual a 0: 11.0


# Experimento 3

## Parte 1
Aplicando o método da similaridade nos 30%

### Similaridade

In [448]:
def vetor_frases_ft_teste(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.get_word_vector(palavra)
                except:
                    soma=soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [449]:
modelo_selecionado = model_ft_cbow

In [450]:
abstract_teste        = {k: preprocessa_abstract(v) for k, v in papers_teste2.items()}
abstract_frases_teste = {k: separa_frases(j) for k, v in abstract_teste.items() for j in v}
abstract_frases_teste = segundo_preprocessamento(abstract_frases_teste)
abstract_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in abstract_frases_teste.items()}

introducao_teste = {k: preprocessa_intro(v) for k, v in papers_teste2.items()}
introducao_frases_teste = {k: separa_frases(j) for k, v in introducao_teste.items() for j in v}
introducao_frases_teste = segundo_preprocessamento(introducao_frases_teste)
introducao_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in introducao_frases_teste.items()}

conclusao_teste = {k: preprocessa_conclusion(v) for k, v in papers_teste2.items()}
conclusao_frases_teste = {k: separa_frases(j) for k, v in conclusao_teste.items() for j in v}
conclusao_frases_teste = segundo_preprocessamento(conclusao_frases_teste)
conclusao_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in conclusao_frases_teste.items()}

### ROUGE-N

#### model_ft_cbow

In [451]:
def get_frases_objetivo_teste(indices_objetivo, idx_paper):
    """retorna a frase objetivo [do abstract] a partir dos indices"""
    frase_objetivo={}
    for idx,idx_f,idx_ind in zip(idx_paper,range(len(list(abstract_frases_teste.values()))),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(list(abstract_frases_teste.values())[idx_f][j])
        frase_objetivo[idx] = lista_one
    return frase_objetivo

In [452]:
v_abstract_teste    = vetor_frases_ft(abstract_palavras_teste,model_ft_cbow)
v_introducao_teste  = vetor_frases_ft(introducao_palavras_teste,model_ft_cbow)
v_conclusao_teste   = vetor_frases_ft(conclusao_palavras_teste,model_ft_cbow)

#Abtract e Introdução
sim_ab_intro_teste,indices_ab_intro_teste,lista_idx_ab_intro_teste =aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_introducao_teste.values()))

#Abtract e Conclusão
sim_ab_conc_teste,indices_ab_conc_teste,lista_idx_ab_c_teste = aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_conclusao_teste.values()))

#Pega os Indices em comum
indices_objetivo_teste = frases_interseccao(lista_idx_ab_intro_teste,lista_idx_ab_c_teste)

#Retorna as Frases correspondentes
frase_objetivo_teste  = get_frases_objetivo_teste(indices_objetivo_teste, papers_teste.keys())

In [453]:
p_ft_cbow_teste, r_ft_cbow_teste, f_ft_cbow_teste  =calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

#w2v
rouge_teste = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

maximo_rouge_por_paper_ft_teste   = max_rouge(rouge_teste)


precisao_sg_ft_teste, revocacao_sg_ft_teste = rouge_precisao_revocacao(maximo_rouge_por_paper_ft_teste, lista_padrao_ouro_teste)
qtd_precisao_ft_teste, qtd_zero_ft_teste, media_ft_teste = analise_precisao(precisao_sg_ft_teste, lista_padrao_ouro_teste)
print('------')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_sg_ft_teste, lista_padrao_ouro_teste, media_ouro_teste)

#### model_ft_sg

In [454]:
v_abstract_teste    = vetor_frases_ft(abstract_palavras_teste,model_ft_sg)
v_introducao_teste  = vetor_frases_ft(introducao_palavras_teste,model_ft_sg)
v_conclusao_teste   = vetor_frases_ft(conclusao_palavras_teste,model_ft_sg)

#Abtract e Introdução
sim_ab_intro_teste,indices_ab_intro_teste,lista_idx_ab_intro_teste =aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_introducao_teste.values()))

#Abtract e Conclusão
sim_ab_conc_teste,indices_ab_conc_teste,lista_idx_ab_c_teste = aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_conclusao_teste.values()))

#Pega os Indices em comum
indices_objetivo_teste = frases_interseccao(lista_idx_ab_intro_teste,lista_idx_ab_c_teste)

#Retorna as Frases correspondentes
frase_objetivo_teste  = get_frases_objetivo_teste(indices_objetivo_teste, papers_teste.keys())


In [455]:
p_ft_sg_teste, r_ft_sg_teste, f_ft_sg_teste  =calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))



#w2v
rouge_teste = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

maximo_rouge_por_paper_ft_teste   = max_rouge(rouge_teste)


precisao_sg_ft_teste, revocacao_sg_ft_teste = rouge_precisao_revocacao(maximo_rouge_por_paper_ft_teste, lista_padrao_ouro_teste)
qtd_precisao_ft_teste, qtd_zero_ft_teste, media_ft_teste = analise_precisao(precisao_sg_ft_teste, lista_padrao_ouro_teste)
print('------')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_sg_ft_teste, lista_padrao_ouro_teste, media_ouro_teste)

#### model_w2v_sg

In [456]:
v_abstract_teste    = vetor_frases_w2v(abstract_palavras_teste, model_w2v_sg)
v_introducao_teste  = vetor_frases_w2v(introducao_palavras_teste, model_w2v_sg)
v_conclusao_teste   = vetor_frases_w2v(conclusao_palavras_teste, model_w2v_sg)

#Abtract e Introdução
sim_ab_intro_teste,indices_ab_intro_teste,lista_idx_ab_intro_teste =aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_introducao_teste.values()))

#Abtract e Conclusão
sim_ab_conc_teste,indices_ab_conc_teste,lista_idx_ab_c_teste = aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_conclusao_teste.values()))

#Pega os Indices em comum
indices_objetivo_teste = frases_interseccao(lista_idx_ab_intro_teste,lista_idx_ab_c_teste)

#Retorna as Frases correspondentes
frase_objetivo_teste  = get_frases_objetivo_teste(indices_objetivo_teste, papers_teste.keys())


In [457]:
p_w2v_sg_teste, r_w2v_sg_teste, f_w2v_sg_teste  = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))


rouge_teste = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

maximo_rouge_por_paper_ft_teste   = max_rouge(rouge_teste)


precisao_sg_ft_teste, revocacao_sg_ft_teste = rouge_precisao_revocacao(maximo_rouge_por_paper_ft_teste, lista_padrao_ouro_teste)
qtd_precisao_ft_teste, qtd_zero_ft_teste, media_ft_teste = analise_precisao(precisao_sg_ft_teste, lista_padrao_ouro_teste)
print('------')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_sg_ft_teste, lista_padrao_ouro_teste, media_ouro_teste)

In [458]:
frase_objetivo_teste

{746: ['this paper presents an unsupervised learning algorithm for sense disambiguation that when trained on unannotated english text rivals the performance of supervised techniques that require timeconsuming hand annotations',
  'the algorithm is based on two powerful constraints that words tend to have one sense per discourse and one sense per collocation exploited in an iterative bootstrapping procedure'],
 747: ['to attack these problems we have built a hybrid generator in which gaps in symbolic knowledge are filled by statistical methods',
  'we also discuss how the hybrid generation model can be used to simplify current generators and enhance their portability even when perfect knowledge is in principle obtainable'],
 748: ['this work is based on the following premises grammars are too complex and detailed to develop manually for most interesting domains parsing models must rely heavily on lexical and contextual information to analyze sentences accurately and existing ngrain mode

#### model_w2v_cbow

In [459]:
v_abstract_teste    = vetor_frases_w2v(abstract_palavras_teste, model_w2v_cbow)
v_introducao_teste  = vetor_frases_w2v(introducao_palavras_teste, model_w2v_cbow)
v_conclusao_teste   = vetor_frases_w2v(conclusao_palavras_teste, model_w2v_cbow)

#Abtract e Introdução
sim_ab_intro_teste,indices_ab_intro_teste,lista_idx_ab_intro_teste =aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_introducao_teste.values()))

#Abtract e Conclusão
sim_ab_conc_teste,indices_ab_conc_teste,lista_idx_ab_c_teste = aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_conclusao_teste.values()))

#Pega os Indices em comum
indices_objetivo_teste = frases_interseccao(lista_idx_ab_intro_teste,lista_idx_ab_c_teste)

#Retorna as Frases correspondentes
frase_objetivo_teste  = get_frases_objetivo_teste(indices_objetivo_teste, papers_teste.keys())

In [460]:
p_w2v_cbow_teste, r_w2v_cbow_teste, f_w2v_cbow_teste  =calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))


#w2v
rouge_teste = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

maximo_rouge_por_paper_ft_teste   = max_rouge(rouge_teste)


precisao_sg_ft_teste, revocacao_sg_ft_teste = rouge_precisao_revocacao(maximo_rouge_por_paper_ft_teste, lista_padrao_ouro_teste)
qtd_precisao_ft_teste, qtd_zero_ft_teste, media_ft_teste = analise_precisao(precisao_sg_ft_teste, lista_padrao_ouro_teste)
print('------')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_sg_ft_teste, lista_padrao_ouro_teste, media_ouro_teste)

### geral

In [813]:
print('w2v_sg_teste')
print('precisão:',media2(media(max_rouge(p_w2v_sg_teste))))
print('revocação:',media2(media(max_rouge(r_w2v_sg_teste))))
print('f-score:',media2(media(max_rouge(f_w2v_sg_teste))))
funcao_80_e_0(p_w2v_sg_teste,r_w2v_sg_teste)
print('-------')

print('w2v_cbow_teste')
print('precisão:',media2(media(max_rouge(p_w2v_cbow_teste))))
print('revocação:',media2(media(max_rouge(r_w2v_cbow_teste))))
print('f-score:',media2(media(max_rouge(f_w2v_cbow_teste))))
funcao_80_e_0(p_w2v_cbow_teste,r_w2v_cbow_teste)

print('-------')
print('ft_sg_teste')
print('precisão:',media2(media(max_rouge(p_ft_sg_teste))))
print('revocação:',media2(media(max_rouge(r_ft_sg_teste))))
print('f-score:',media2(media(max_rouge(f_ft_sg_teste))))
funcao_80_e_0(p_ft_sg_teste,r_ft_sg_teste)

print('-------')
print('ft_cbow_teste')
print('precisão:',media2(media(max_rouge(p_ft_cbow_teste))))
print('revocação:',media2(media(max_rouge(r_ft_cbow_teste))))
print('f-score:',media2(media(max_rouge(f_ft_cbow_teste))))
funcao_80_e_0(p_ft_cbow_teste,r_ft_cbow_teste)

w2v_sg_teste
precisão: 0.886
revocação: 0.906
f-score: 0.888
precisao acima de 80%: 86.0
recall acima de 80%: 90.0
igual a 0: 7.0
-------
w2v_cbow_teste
precisão: 0.891
revocação: 0.909
f-score: 0.89
precisao acima de 80%: 87.0
recall acima de 80%: 90.0
igual a 0: 5.0
-------
ft_sg_teste
precisão: 0.889
revocação: 0.913
f-score: 0.893
precisao acima de 80%: 86.0
recall acima de 80%: 90.0
igual a 0: 6.0
-------
ft_cbow_teste
precisão: 0.892
revocação: 0.907
f-score: 0.891
precisao acima de 80%: 87.0
recall acima de 80%: 89.0
igual a 0: 6.0


# Extra

In [869]:
palavras_papers_teste2 = {k:[word_tokenize(f) for f in v] for k,v in frases_papers_teste.items()}
palavras_papers_teste_es = funcao_aplicar_es_sentencer(palavras_papers_teste2)
palavras_papers_teste = {k:v for k,v in zip(palavras_papers_teste2.keys(),palavras_papers_teste_es)}

In [870]:
bigramas_teste          = gerador_ngramas(token_frases_teste,2)
probabilidades_bigramas  = gerador_probabilidades(bigramas_teste, model_bi)

bigramas_perp_teste       = funcao_perplexidade(list(frases_papers_teste.values()), 2 , model_bi)
media_bigramas_perp_teste = media_perplexidade(bigramas_perp_teste)

bigramas_perp_teste_s_inf       = lista_sem_infinito(bigramas_perp_teste)
media_bigramas_perp_teste_s_inf = media_lista_sem_infinito(bigramas_perp_teste_s_inf)

frase_objetivo_bigramas_teste    = selecao_frases_obj_mle(probabilidades_bigramas, frases_papers_teste2)
lista_frase_teste_bigramas,lista_prob_teste_bigramas,lista_geral_teste_bigramas = filtro_selecao_frases_mle(frase_objetivo_bigramas_teste)

In [871]:
#lista_prob_frase - guarda a probabilidade de cada frase, seu tamanho é a qtd de frases
#lista_prob_frase_paper - guarda a probabilidade de todas as frases de cada paper, seu tamanho é a qtd de paper

lista_prob_frase_paper_teste=[]
for paper in lista_bigramas2:
    lista_prob_frase=[]
    for frase in paper:
        prob_frase=1
        for bigrama in frase:
            prob = (model_bi.score(bigrama[1], context=[bigrama[0]]))
            prob_frase = prob_frase * prob
            #print(prob)
            #print(trigrama, prob)
        #print('PROB FRASE',prob_frase)
        lista_prob_frase.append(prob_frase)
    lista_prob_frase_paper_teste.append(lista_prob_frase)

#Analisando o tamanho das frases de teste
x=[]
for j in list(frase_objetivo_teste.values()):
    for i in j:
        x.append(len(i.split(' ')))
np.mean(x)

In [872]:
lista=[]
for i,j in zip(range(len(lista_prob_teste_bigramas)),list(frases_papers_teste.keys()) ):
    lista.append(sorted(zip(lista_prob_teste_bigramas[i],frases_papers_teste[j]), reverse=True))
    
todos=[]
for i in range(len(lista)):
    x=[]
    for i in list(lista[i]):
        frases = i[1].split(' ')
        if (len(frases) >=19) and i[0]!=1:
            x.append(i)
    todos.append(x)
    
frases_objetivo_ml_teste=[]
for i in range(len(todos)):
    frases_objetivo_ml_teste.append(todos[i][:3])

In [873]:
frase_objetivo_teste_exp2=[]
for lista_prob_teste_bigramas in frases_objetivo_ml_teste:
    cada_paper=[]
    for frase_obj_prob in lista_prob_teste_bigramas:
        cada_paper.append(frase_obj_prob[1])
    frase_objetivo_teste_exp2.append(cada_paper)

dict(zip(list(frases_papers_teste.keys()),frase_objetivo_teste_exp2))

In [874]:
p_bi, r_bi, f_bi = calculo_rouge(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values()))

In [875]:
print('bigrama com rest')
print('precisão:',media2(media(max_rouge(p_bi))))
print('revocação:',media2(media(max_rouge(r_bi))))
print('f-score:',media2(media(max_rouge(f_bi))))
funcao_80_e_0(p_bi,r_bi)

bigrama com rest
precisão: 0.009
revocação: 0.009
f-score: 0.009
precisao acima de 80%: 1.0
recall acima de 80%: 1.0
igual a 0: 99.0


rouge_total=[]
for i,j in zip(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values())):
    rouge_1={}
    for h in range(len(i)):
        for z in range(len(j)):
            rouge_1[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].precision)
    rouge_total.append(rouge_1)

maximo_rouge_paper = max_rouge(rouge_total)

## trigramas

In [ ]:
def selecao_frases_obj_mle(probabibilidade, frases_papers):
    """função para selecionar até 3 maiores elementos da lista"""
    #Pega os 3 elementos de maior probabilidade
    lista=[]
    dicionario={}
    x=[]
    for i,j in zip(range(len(probabibilidade)),range(len(frases_papers))):
        lista.append(sorted(zip(probabibilidade[i],frases_papers[j]), reverse=True)[:3])
    return lista

In [876]:
if os.path.exists('model_tri'):
    model_tri = joblib.load('model_tri')
else:
    n = 3
    paddedLine = [list(pad_both_ends(word_tokenize(s), n)) for s in frase_objetivo_ft_formato_ml]
    train_data, padded_sents = padded_everygram_pipeline(n, paddedLine)

    model_tri = MLE(n)
    model_tri.fit(train_data, padded_sents)

    print(model_tri.vocab)

In [879]:
trigramas_teste          = gerador_ngramas(token_frases_teste,3)
probabilidades_trigramas = gerador_probabilidades(trigramas_teste, model_tri)


In [911]:
lista=[]
for i,j in zip(range(len(probabilidades_trigramas)),list(frases_papers_teste.keys()) ):
    lista.append(sorted(zip(probabilidades_trigramas[i],frases_papers_teste[j]), reverse=True))

In [912]:
todos=[]
for i in range(len(lista)):
    x=[]
    for i in list(lista[i]):
        frases = i[1].split(' ')
        if (len(frases) >=19) and i[0]!=1:
            x.append(i)
    todos.append(x)
    
frases_objetivo_ml_teste=[]
for i in range(len(todos)):
    frases_objetivo_ml_teste.append(todos[i][:3])

In [915]:
frases_objetivo_ml_teste

[[(0.0,
   'words that cooccur with the target word in unusually great frequency especially in certain collocational relationships will tend to be reliable indicators of one of the target words senses eg flock and bulldozer for crane'),
  (0.0,
   'words not only tend to occur in collocations that reliably indicate their sense they tend to occur in multiple such collocations'),
  (0.0,
   'words in the entry appearing in the most reliable collocational relationships with the target word are given the most weight based on the criteria given in yarowsky')],
 [(0.0,
   'word lattices are commonly used to model uncertainty in speech recognition waibel and lee and are well adapted for use with ngram models'),
  (0.0,
   'while true irregular forms eg child i children must be kept in a small exception table the problem of multiple regular patterns usually increases the size of this table dramatically'),
  (0.0,
   'while this experiment shows that statistical models can help make choices in 

In [882]:
frase_objetivo_teste_exp2=[]
for lista_frase_obj_prob in frases_objetivo_ml_teste:
    cada_paper=[]
    for frase_obj_prob in lista_frase_obj_prob:
        cada_paper.append(frase_obj_prob[1])
    frase_objetivo_teste_exp2.append(cada_paper)

In [883]:
p_r_tri, r_r_tri, f1_r_tri = calculo_rouge(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values()))

rouge_total=[]
for i,j in zip(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values())):
    #print (len(i))
    rouge_1={}
    for h in range(len(i)):
        for z in range(len(j)):
            rouge_1[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].precision)
    rouge_total.append(rouge_1)

maximo_rouge_paper = max_rouge(rouge_total)

precisao_exp3_pt2, revocacao_exp3_pt2 = rouge_precisao_revocacao(maximo_rouge_paper, lista_padrao_ouro_teste)

print('------ revocação ---------')
print('média recall',np.mean(revocacao_exp3_pt2))
qtd_revocacao_exp3_pt2, media_revocacao_exp3_pt2  = analise_revocacao(revocacao_exp3_pt2, lista_padrao_ouro_teste, media_ouro_teste)

print('------ precisão ---------')
qtd_precisao_exp3_pt2, qtd_zero_exp3_pt2, media_exp3_pt2 = analise_precisao(precisao_exp3_pt2, lista_padrao_ouro_teste)

## UNIGRAMAS

In [916]:
unigramas_teste          = gerador_ngramas(token_frases_teste,1)
probabilidades_unigramas  = gerador_probabilidades(unigramas_teste, model_uni)

In [917]:
lista=[]
for i,j in zip(range(len(probabilidades_unigramas)),list(frases_papers_teste.keys()) ):
    lista.append(sorted(zip(probabilidades_unigramas[i],frases_papers_teste[j]), reverse=True))

In [918]:
todos=[]
for i in range(len(lista)):
    x=[]
    for i in list(lista[i]):
        frases = i[1].split(' ')
        if (len(frases) >=19) and i[0]!=1:
            x.append(i)
    todos.append(x)
    
frases_objetivo_ml_teste=[]
for i in range(len(todos)):
    frases_objetivo_ml_teste.append(todos[i][:3])

In [919]:
frase_objetivo_teste_exp2=[]
for lista_frase_obj_prob in frases_objetivo_ml_teste:
    cada_paper=[]
    for frase_obj_prob in lista_frase_obj_prob:
        cada_paper.append(frase_obj_prob[1])
    frase_objetivo_teste_exp2.append(cada_paper)

In [920]:
frases_objetivo_ml_teste

[[(1.1093008973424372e-49,
   'for each possible sense of the word identify a relatively small number of training examples representative of that sense'),
  (8.277624632415869e-61,
   'this would indicate that the cost of a large sensetagged training corpus may not be necessary to achieve accurate wordsense disambiguation'),
  (2.644463252716067e-64,
   'words not only tend to occur in collocations that reliably indicate their sense they tend to occur in multiple such collocations')],
 [(3.2209589774547606e-57,
   'while this experiment shows that statistical models can help make choices in generation it fails as a computational strategy'),
  (0.0,
   'word lattices are commonly used to model uncertainty in speech recognition waibel and lee and are well adapted for use with ngram models'),
  (0.0,
   'while true irregular forms eg child i children must be kept in a small exception table the problem of multiple regular patterns usually increases the size of this table dramatically')],
 

In [888]:
dict(zip(list(frases_papers_teste.keys()),frase_objetivo_teste_exp2))

{746: ['for each possible sense of the word identify a relatively small number of training examples representative of that sense',
  'this would indicate that the cost of a large sensetagged training corpus may not be necessary to achieve accurate wordsense disambiguation',
  'words not only tend to occur in collocations that reliably indicate their sense they tend to occur in multiple such collocations'],
 747: ['while this experiment shows that statistical models can help make choices in generation it fails as a computational strategy',
  'word lattices are commonly used to model uncertainty in speech recognition waibel and lee and are well adapted for use with ngram models',
  'while true irregular forms eg child i children must be kept in a small exception table the problem of multiple regular patterns usually increases the size of this table dramatically'],
 748: ['it remains to be shown that an accurate broadcoverage parser can improve the performance of a text processing applica

In [889]:
p_uni, r_uni, f1_uni = calculo_rouge(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values()))

rouge_total=[]
for i,j in zip(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values())):
    #print (len(i))
    rouge_1={}
    for h in range(len(i)):
        for z in range(len(j)):
            rouge_1[(h,z)] = (scorer.score(j[z],i[h])['rouge1'].precision)
    rouge_total.append(rouge_1)

maximo_rouge_paper = max_rouge(rouge_total)

precisao_exp3_pt2, revocacao_exp3_pt2 = rouge_precisao_revocacao(maximo_rouge_paper, lista_padrao_ouro_teste)

lista=[]
for i,j in zip(range(len(probabilidades_unigramas)),list(frases_papers_teste.keys()) ):
    lista.append(sorted(zip(probabilidades_unigramas[i],frases_papers_teste[j]), reverse=True))

frases_papers_teste

frases_papers_teste[773]

for i,j in zip(range(len(maximo_rouge_paper)),frases_papers_teste.keys()):
    print(i,j)

len(maximo_rouge_paper[0])

x=[]
for i in range(len(maximo_rouge_paper)):
    for j in range(3):
        x.append(maximo_rouge_paper[i][j][1])

np.mean(x)

for i,j in zip(frases_papers_teste.keys(),maximo_rouge_paper):
    print(i,j)

dict(zip(list(frases_papers_teste.keys()),frase_objetivo_teste_exp2))

dict(zip(list(frases_papers_teste.keys()),frase_objetivo_teste_exp2))

print('------ revocação ---------')
print('média recall',np.mean(revocacao_exp3_pt2))
qtd_revocacao_exp3_pt2, media_revocacao_exp3_pt2  = analise_revocacao(revocacao_exp3_pt2, lista_padrao_ouro_teste, media_ouro_teste)

print('------ precisão ---------')
qtd_precisao_exp3_pt2, qtd_zero_exp3_pt2, media_exp3_pt2 = analise_precisao(precisao_exp3_pt2, lista_padrao_ouro_teste)

#Analisando o tamanho das frases de teste
x=[]
for j in list(padrao_ouro_treino_frases.values()):
    for i in j:
        x.append(len(i.split(' ')))
np.mean(x)

#Extra
#for i in lista[0]:
    #frases = i[1].split(' ')
    #if len(frases) >=27:
        #print(i)
        
#

padrao_ouro_teste_frases[765]

print('unigrama com rest')
print('precisão:',media2(media(max_rouge(p_uni))))
print('revocação:',media2(media(max_rouge(r_uni))))
print('f-score:',media2(media(max_rouge(f1_uni))))
funcao_80_e_0(p_uni,r_uni)

print('-----')
print('bigrama com rest')
print('precisão:',media2(media(max_rouge(p_bi))))
print('revocação:',media2(media(max_rouge(r_bi))))
print('f-score:',media2(media(max_rouge(f_bi))))
funcao_80_e_0(p_bi,r_bi)

print('-----')
print('trigrama com rest')
print('precisão:',media2(media(max_rouge(p_r_tri))))
print('revocação:',media2(media(max_rouge(r_r_tri))))
print('f-score:',media2(media(max_rouge(f1_r_tri))))
funcao_80_e_0(p_r_tri,r_r_tri)

## bigrama

In [929]:
import statistics

In [930]:
x=[]
for i in range(len(list(padrao_ouro_teste_frases.values()))):
    for j in list(padrao_ouro_teste_frases.values())[i]:
        x.append(len(j.split(' ')))

In [931]:
y=[]
for i in range(len(list(padrao_ouro_treino_frases.values()))):
    for j in list(padrao_ouro_treino_frases.values())[i]:
        y.append(len(j.split(' ')))

In [932]:
np.average(x+y)

19.62478452403754

In [933]:
statistics.mode(x)

20

In [934]:
statistics.mode(y)

18

In [935]:
statistics.mode(x+y)

18

In [936]:
y=[]
for i in range(len(list(padrao_ouro_treino_frases.values()))):
    for j in list(padrao_ouro_treino_frases.values())[i]:
        y.append(len(j.split(' ')))

In [937]:
np.min(y)

2

In [938]:
x=[]
for i in list(padrao_ouro_teste_frases.values()):
    x.append(len(i))

In [939]:
y=[]
for i in list(padrao_ouro_treino_frases.items()):
    y.append(len(i))

In [940]:
np.mean(x)+np.mean(y)

8.783018867924529

In [941]:
bigramas_teste          = gerador_ngramas(token_frases_teste,2)
probabilidades_bigramas  = gerador_probabilidades(bigramas_teste, model_bi)

In [942]:
probabilidades_bigramas==probabilidades_trigramas

False

In [943]:
lista=[]
for i,j in zip(range(len(probabilidades_bigramas)),list(frases_papers_teste.keys()) ):
    lista.append(sorted(zip(probabilidades_bigramas[i],frases_papers_teste[j]), reverse=True))

In [944]:
todos=[]
for i in range(len(lista)):
    x=[]
    for i in list(lista[i]):
        frases = i[1].split(' ')
        if (len(frases) >=19) and i[0]!=1:
            x.append(i)
    todos.append(x)
    

In [945]:
frases_objetivo_ml_teste=[]
for i in range(len(todos)):
    frases_objetivo_ml_teste.append(todos[i][:3])

In [946]:
frases_objetivo_ml_teste

[[(0.0,
   'words that cooccur with the target word in unusually great frequency especially in certain collocational relationships will tend to be reliable indicators of one of the target words senses eg flock and bulldozer for crane'),
  (0.0,
   'words not only tend to occur in collocations that reliably indicate their sense they tend to occur in multiple such collocations'),
  (0.0,
   'words in the entry appearing in the most reliable collocational relationships with the target word are given the most weight based on the criteria given in yarowsky')],
 [(0.0,
   'word lattices are commonly used to model uncertainty in speech recognition waibel and lee and are well adapted for use with ngram models'),
  (0.0,
   'while true irregular forms eg child i children must be kept in a small exception table the problem of multiple regular patterns usually increases the size of this table dramatically'),
  (0.0,
   'while this experiment shows that statistical models can help make choices in 

In [947]:
frase_objetivo_teste_exp2=[]
for lista_frase_obj_prob in frases_objetivo_ml_teste:
    cada_paper=[]
    for frase_obj_prob in lista_frase_obj_prob:
        cada_paper.append(frase_obj_prob[1])
    frase_objetivo_teste_exp2.append(cada_paper)

In [948]:
p_bi, r_bi, f_bi = calculo_rouge(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values()))

In [949]:
print('unigrama com rest')
print('precisão:',media2(media(max_rouge(p_uni))))
print('revocação:',media2(media(max_rouge(r_uni))))
print('f-score:',media2(media(max_rouge(f1_uni))))
funcao_80_e_0(p_uni,r_uni)

print('-----')
print('bigrama com rest')
print('precisão:',media2(media(max_rouge(p_bi))))
print('revocação:',media2(media(max_rouge(r_bi))))
print('f-score:',media2(media(max_rouge(f_bi))))
funcao_80_e_0(p_bi,r_bi)

print('-----')
print('trigrama com rest')
print('precisão:',media2(media(max_rouge(p_r_tri))))
print('revocação:',media2(media(max_rouge(r_r_tri))))
print('f-score:',media2(media(max_rouge(f1_r_tri))))
funcao_80_e_0(p_r_tri,r_r_tri)

unigrama com rest
precisão: 0.344
revocação: 0.401
f-score: 0.336
precisao acima de 80%: 0.0
recall acima de 80%: 1.0
igual a 0: 0.0
-----
bigrama com rest
precisão: 0.273
revocação: 0.376
f-score: 0.282
precisao acima de 80%: 0.0
recall acima de 80%: 1.0
igual a 0: 0.0
-----
trigrama com rest
precisão: 0.273
revocação: 0.376
f-score: 0.282
precisao acima de 80%: 0.0
recall acima de 80%: 1.0
igual a 0: 0.0
